# Reference Update

This notebook is an implementation, debugging and analysis of a Reference Update Process using *CoppeliaSim* as a renderer. 

The goal is to implement and analyse, how well the method implemented will perform on estimating the new reference. For this:
1. Make *CoppeliaSim's* clients that detects and sends blob coordinates detected in Vision Sensors' images;
2. Receive the messages asynchronously;
3. Triangulate the marker positions;
4. Label the markers in the reference wand for each frame and client;
5. Estimate the new reference based in the wand's x and y directions.

---

In [1]:
# Importing modules...
import numpy as np
import scipy as sp
from scipy.optimize import linear_sum_assignment
import socket

import sys
sys.path.append('../..') # Go back to base directory

from modules.plot.viewer3d import Viewer3D

from modules.vision.camera import Camera
from modules.vision.synchronizer import Synchronizer

from modules.integration.client import Client
from modules.integration.coppeliasim.server import CoppeliaSim_Server

# Instanciating `Server` and `Client` Structures

To wrap the information of all clients and mediate the communication between this notebook and *CoppeliaSim*, a `Server` object will be instanciated. 

The `Client` objects will be generated by their camera model and a synchronizer, representing their respective twin in the simulation.

For the `Server` instanciation, the following parameters must be given:
- Server address;
- *CoppeliaSim's* simulation Controller address;
- List containing all the clients present in the scene.

---

In [2]:
n_clients = 4 # Number of clients in the arena
clients = []  # Clients list

# Object matrix of Camera 0
base_matrix = np.array([[-7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 7.07106781e-01,  5.00000000e-01, -5.00000000e-01, 2.50000000e+00],
                        [ 1.46327395e-13, -7.07106781e-01, -7.07106781e-01, 2.50000000e+00]])

# Create clients
for ID in range(n_clients):
    # Spread all cameras uniformely in a circle around the arena
    R = np.array(sp.spatial.transform.Rotation.from_euler('z', (360 / n_clients) * ID, degrees=True).as_matrix())
    pose = np.vstack((R @ base_matrix,
                      np.array([0, 0, 0, 1])))

    # Generate associated camera model
    camera = (Camera(# Intrinsic Parameters
                     resolution=(1080,1080), 
                     fov_degrees=60.0,     
     
                     # Camera Pose
                     pose=pose,
     
                     # Rational Lens Distortion Model
                     # distortion_model='rational',
                     # distortion_coefficients=np.array([0.014, -0.003, -0.0002, -0.000003, 0.0009, 0.05, -0.007, 0.0017]),
 
                     # Fisheye Lens Distortion Model
                     distortion_model='fisheye',
                     distortion_coefficients=np.array([0.395, 0.633, -2.417, 2.110]),
     
                     # Image Noise Model
                     snr_dB=13
                     ))
    
    clients.append(Client(camera=camera))

In [3]:
# Create server
server = CoppeliaSim_Server(clients=clients,
                            server_address=('127.0.0.1', 8888),
                            controller_address=('127.0.0.1', 7777))

# Requesting Scene

A **Scene Request** will send to *CoppeliaSim* the data necessary to instanciate the simulated clients' twins in the childscripts. The scene request can be used to reset the client's data from the server if called again.

---

In [4]:
# Request scene with the associated server clients
if not server.request_scene():
    sys.exit() # Scene request failed!

[SERVER] Wrapping up CoppeliaSim scene info
[SERVER] Scene info sent
[SERVER] Scene set!


# Requesting Capture

A **Capture Request** will trigger a simulation in *CoppeliaSim* sending the total simulation time of the requested capture. Once the simulation stops, another capture request can be called for another simulation

In the simulation start, the clients will be created and send their ID to the server for client registration.

---

In [5]:
# Capture specifications
blob_count = 3 # Number of expected markers
capture_time = 1.0 # In seconds
window = 3 # The minimum ammount of points for interpolating 
throughput = 20 # Triangulated scenes per second
step = 1 / throughput # Interpolation timestep

# Capture synchronizer
synchronizer = Synchronizer(blob_count, window, step, capture_time)

# Request capture (start simulation)
if not server.request_capture(synchronizer):
    sys.exit() # Capture request failed!

# Wait for client identification
server.register_clients()

[SERVER] Capture info sent
[SERVER] Capture confirmed!
[SERVER] Waiting for clients...
	Client 0 registered
	Client 1 registered
	Client 2 registered
	Client 3 registered
[SERVER] All clients registered!


# Running Simulation

The messages will be received here in the following loop until a server timeout is reached. To analyse the content of each message, toggle the `verbose` flag. 

The loop will wait for a message to be received by the socket. When a message comes, it will be recorded in their client's `message_log` and wait for the next message. 

Since the calibration prioritizes the amount of quality data and not real time triangulation, the in-loop actions will be post-processed to avoid message losses.

---

In [6]:
verbose = False

timeout = 5 # In seconds
server.udp_socket.settimeout(timeout) # Set server timeout
print(f'[SERVER] Timeout set to {timeout} seconds\n')

# Breaks in the timeout
while True: 
    # Wait for message - Event guided!
    try:
        message_bytes, address = server.udp_socket.recvfrom(server.buffer_size)

    except socket.timeout as err:
        print('\n[SERVER] Timed Out!')
        
        break # Close capture loop due to timeout
    
    # Check if client exists
    try:
        ID = server.client_addresses[address] # Client Identifier
    
    except:
        if verbose: print('> Client not recognized')

        continue # Jump to wait for the next message
    
    # Show sender
    if verbose: print(f'> Received message from Client {ID} ({address[0]}, {address[1]})')

    # Save message
    server.clients[ID].message_log.append(message_bytes)

[SERVER] Timeout set to 5 seconds


[SERVER] Timed Out!


# Post-processing Data

For each client, the code will loop through it's message history and it will:
1. Decode message; 
2. Parse the message for it's contents;
3. Check if the message is valid;
    - A valid message is composed of a blob coodinate and it's area (per blob) and the PTS of the message.
4. Undistort blob data
5. Save data in the `Synchronizer` structure.

---

In [7]:
verbose = False

for client in server.clients:
    # Parse through client's message history
    for message_bytes in client.message_log: 

        # Decode message
        try:
            message = np.frombuffer(message_bytes, dtype=np.float32)

        except:
            if verbose: print('> Couldn\'t decode message')

            continue # Jump to the next message

        # Empty message
        if not message.size:
            if verbose: print('\tEmpty message')

            continue # Jump to the next message

        # Extracting the message's PTS
        PTS = message[-1] # Last element of the message 

        # Valid message is [u, v, A] per blob and the PTS of the message
        if message.size !=  3 * blob_count + 1:

            if message.size == 1: # Only PTS
                if verbose: print(f'\tNo blobs were detected - {PTS :.3f} s')

            else: 
                if verbose: 
                    print(f'\tWrong blob count or corrupted message')
                    print(f'Corrupted Message: {message}')

            continue # Jump to the next message

        # Extracting blob data (coordinates & area)
        sync_blobs_pair = message[:-1].reshape(-1, 3) # All but last element (reserved for PTS)

        # Extracting centroids
        blob_centroids = sync_blobs_pair[:,:2] # Ignoring their area

        # Undistorting blobs centroids
        undistorted_blobs = client.camera.undistort_points(blob_centroids)          

        # Print blobs
        if verbose:
            print(f'\tDetected Blobs - {PTS :.3f} s')
            print('\t' + str(sync_blobs_pair).replace('\n', '\n\t'))

        # Save data
        valid_data = client.synchronizer.add_data(undistorted_blobs, PTS)

        if verbose: 
            if valid_data:
                print('\tData Accepted!')
            else:
                print('\tData Refused!')

# Reference Update

---

In [8]:
def perpendicular_order(markers, wand_ratio):
    # Distances between markers
    distances = np.array([np.linalg.norm(markers[0] - markers[1]), 
                          np.linalg.norm(markers[1] - markers[2]), 
                          np.linalg.norm(markers[2] - markers[0])])
    
    # Shortest distance
    shortest = np.min(distances)

    # If x is invalid
    if shortest == 0.0 or shortest is np.nan:
        return None

    # Normalize distances
    distances /= shortest

    # Measured unique distance sums
    measured_unique_sums = np.array([distances[0] + distances[2],
                                     distances[0] + distances[1],
                                     distances[1] + distances[2]])
    
    # Expected unique distance sums
    expected_unique_sums = np.array([wand_ratio[0] + wand_ratio[1],
                                     wand_ratio[1] + np.sqrt(wand_ratio[0]**2 + wand_ratio[1]**2),
                                     wand_ratio[0] + np.sqrt(wand_ratio[0]**2 + wand_ratio[1]**2)])
    
    # Error matrix
    difference_matrix = np.array([[np.abs(measured - expected)
                                   for measured in measured_unique_sums] 
                                   for expected in expected_unique_sums])

    # Using the hungarian (Munkres) assignment algorithm to find unique correspondences between blobs and epilines
    _, new_indices = linear_sum_assignment(difference_matrix)

    return markers[new_indices]

# Ratio between perpendicularly matched marker distances
wand_ratio = (1.0, 2.0)

In [13]:
# Find transformation to align point-set A to point-set B
def kabsch(A, B):
    assert A.shape == B.shape

    # Find centroids and ensure they are 3x1
    A_c = np.mean(A, axis=1).reshape(-1, 1)
    B_c = np.mean(B, axis=1).reshape(-1, 1)

    # Centralize point-sets in origin
    A_0 = A - A_c
    B_0 = B - B_c

    H = A_0 @ np.transpose(B_0)

    # Find rotation using Singular Value Decomposition
    U, _, Vt = np.linalg.svd(H)
    R = Vt.T @ U.T

    # Special reflection case
    if np.linalg.det(R) < 0: # The rotation matrix determinant should be +1
        Vt[2,:] *= -1
        R = Vt.T @ U.T

    # Finding translation vector
    t = -R @ A_c + B_c

    return np.vstack((np.hstack((R, t)), np.array([0, 0, 0, 1])))

In [65]:
pair = (0, 1)
reference, auxiliary = pair

A, B, C = [], [], []

for sync_blobs in zip(*[server.clients[ID].synchronizer.async_blobs for ID in pair]):
    triangulated_markers = server.multiple_view.triangulate_by_pair(pair, [sync_blobs[0], sync_blobs[1]])
    ordered_triangulated_markers = perpendicular_order(triangulated_markers.T, wand_ratio)

    A.append(ordered_triangulated_markers[0])
    B.append(ordered_triangulated_markers[1])
    C.append(ordered_triangulated_markers[2])

A = np.mean(np.array(A), axis=0).reshape(3, -1)
B = np.mean(np.array(B), axis=0).reshape(3, -1)
C = np.mean(np.array(C), axis=0).reshape(3, -1)

x = (B - A) / np.linalg.norm(B - A) 
y = (C - A) / np.linalg.norm(C - A) 
z = np.cross(np.concatenate(x), np.concatenate(y)).reshape(3, -1) 

estimated = np.hstack((x * 1, y * 2, z * 3)) + A
canon = np.hstack((np.array([[1], [0], [0]]),
                   np.array([[0], [2], [0]]),
                   np.array([[0], [0], [3]])))

transformation = kabsch(estimated, canon)
new_reference = np.linalg.inv(transformation)

In [66]:
# Create the Scene Viewer
scene = Viewer3D(title='Calibrated Camera Poses', 
                 size=10)

# Add camera frames to the scene
for ID, camera in enumerate(server.multiple_view.camera_models): 
    scene.add_frame(camera.pose, f'Camera {ID}', axis_size=0.4)

# Add triangulated markers to the scene
scene.add_points(A, 'A')
scene.add_points(B, 'B')
scene.add_points(C, 'C')

scene.add_frame(np.eye(4), 'World Frame', axis_size=0.4)
scene.add_frame(new_reference, 'New Frame', axis_size=0.4)

# Plot scene
scene.figure.show(renderer='notebook_connected')